<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 12pt;"><strong>Economic Models</strong>, Fall 2019<br>
            Dr. Eric Van Dusen</p></td></tr>
</table>

# Gender Bias in an Economics Academic Setting

Alice Wu's two papers "Gender Stereotyping in Academia: Evidence from Economics Job
Market Rumors Forum" and "Gendered Language on the Economics Job Market Rumors Forum" explore how women and men are portrayed differently anonymously in Economics academia. By analyzing Economics Job Market Rumors forum (EJMR), an online Economics forum for Economics graduate students, Wu finds clear differences between how both men and women are discussed. Using a Lasso-Logistic model to measure the gendered langauge in the posts and threads on EJMR, Wu identifies words that are most associated with each gender. Wu finds that words that are most predictive of women are about physical apperance or personal information, while words that are most predictive of men focus on academic or professional qualities. 

In this lab, we will go through the end-to-end process Wu implemented to arrive at her conclusions. This is a combination of code and the PowerPoint presented during class. 

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV

# Step 1: Train and Test Data

We will first work with two datasets: "gendered_posts.csv" and "keys_to_X.csv". The former contains posts and their respective topics. The latter contains each post's ID and its topic's ID. Because we are working with keys for each X, we need to make sure that we are working with the corresponding key for each X. We will identify the training and test samples that are stored within the two CSV files. 

In [ ]:
dir_data="../" # specify the directory to data files 
dir_lasso="../" # where the outputs are saved 

In [ ]:
posts=pd.read_csv("/data/shared/temporary/pandp.20181101/gendered_posts.csv")
keys_X=pd.read_csv('keys_to_X.csv')

In [ ]:
posts.head()

In [ ]:
keys_X.head()

We will now ensure that the order is consistent with the matrix "X" of word counts by merging the two datasets. If each dataset has a different number of rows, then it will not merge correctly. Below, we see that it did merge correctly, indicating that the order of data is consistent across each dataset. Now, we can see all information according to each post and its title ID without having to switch between each dataset. 

In [ ]:
keys_merged=pd.merge(keys_X,posts,on=['title_id','post_id'],how="left")
keys_merged.shape

In keys_merged, we see that there are 444810 rows with 16 columns. For the purpose of this exercise, we will shorten the data to only 20% of the sample size. We will choose the top 20% rows of the data to proceed. 

In [ ]:
keys_merged_sample = keys_merged.iloc[:88962]
keys_merged_sample.shape

In machine learning, we divide the dataset into two parts: training set and test set. The training set is the subset that trains the model, while the test set is the subset that tests the trained model. The training set is the actual dataset that trains the model; the model sees and learns from the training data. The test set evaluates the model on new data that the training model has not seen before. When splitting the data into two subsets, it is important to not cherrypick the data-- the test set should be representative of the training set and vice versa. If the two subsets differ in terms of characteristics, then the model will not work properly and will not generalize well to the test data and even new data. A common split is a 75 - 25 split, where 75% of the data is used for training, and 25% is used for testing. 

In this case, we are splitting the data such that the training subset will be named i_train, and there are two testing subsets called i_test0 and i_test1. 

In [ ]:
i_train=np.where(keys_merged_sample['training_pronoun']==1) 
i_test0=np.where(keys_merged_sample['training_pronoun']==0) 
i_test1=np.where((keys_merged_sample['fem_pronoun']>0) & (keys_merged_sample['male_pronoun']>0))  

The training data for ouput is as follows

In [ ]:
y_train=keys_merged_sample.loc[i_train[0],'female_pronoun'].as_matrix()

# Step 2: Bring in word count for matrix X

In [ ]:
word_counts=np.load("/data/shared/temporary/pandp.20181101/X_word_count.npz",encoding='latin1', allow_pickle=True)
X=word_counts['X'][()] 
X_train=X[i_train[0],:]
X_test0=X[i_test0[0],:]
X_test1=X[i_test1[0],:]

# Step 3: Select Predictors: most frequent 10K excluding gender classifiers & additional last names

In [ ]:
vocab10K=pd.read_csv("vocab10K.csv")
vocab10K['exclude'].sum() 
exclude_vocab=vocab10K.loc[vocab10K['exclude']==1,:]
i_exclude=exclude_vocab['index']-1 

i_columns=range(10000)
i_keep_columns=list(set(i_columns)-set(i_exclude)) 
np.savetxt(dir_lasso+"i_keep_columns.txt",i_keep_columns)  

X_train=X_train[:,i_keep_columns] 
print(X_train.shape)              
X_test0=X_test0[:,i_keep_columns] 
print(X_test0.shape)              
X_test1=X_test1[:,i_keep_columns] 
print(X_test1.shape)

# Linear LASSO Pronoun Sample Model

Linear regression models are supervised machine learning models that predict output using a linear function as follows 

${Yi = B0 + B1Xi + B2Xi}$

In this case, ${Yi}$ is the dependent variable, ${B0}$ is the Y-intercept, ${B1}$ is the slope coefficient for feature 1 and ${Xi}$ is the independent variable for feature 1,  ${B2}$ is the slope coefficient for feature 2 and ${Xi}$ is the independent variable for feature 2. When working with linear models, it is important to choose a model that does not overfit or underfit the data. 

If there are many features and the test score is less than the training score, then the model might be over-generalizing or over-fitting. Often, if a dataset has too few of features and has a poor training and test score, then there might be under-fitting. 

Lasso regression models can reduce the model complexity and avoid over-fitting by implementing a cost function that takes the coefficients' absolute values into account. This is called L1 loss regularization. 

The two most common types of loss functions for linear models are L1 and L2 loss. L1 loss minimizes the sum of the absolute differnces between the predicted and actual values. L2 loss minimizes the sum of the square of differneces between the predicted and actual values. Based on the scenario, data scientists need to choose whether to implement L1 loss function or L2 loss function. In this case, we will use L1 loss function, as it relates to the LASSO model.

The linear LASSO model takes a while to run. For the purpose of this exercise, we will briefly conceptually discuss the model but not run it during class. 

In [ ]:
model=LassoCV(cv=5).fit(X_train,y_train)
coef=model.coef_
np.savetxt("coef_lasso_linear_pronoun.txt",coef) 

We will now predict the probability of a post being Female. 

In [ ]:
ypred_train=model.predict_proba(X_train)[:,1] 
ypred_test0=model.predict_proba(X_test0)[:,1]
ypred_test1=model.predict_proba(X_test1)[:,1]

np.savetxt("linear_ypred_train.txt",ypred_train)
np.savetxt("linear_ypred_test0.txt",ypred_test0)
np.savetxt("linear_ypred_test1.txt",ypred_test1)

# Logistic Classification Full Sample Model

Logistic classification is a classification model that categorizes data across different topics. In this case, we are working with a classification model that classifies a post as catered towards either men or women. Assuming that ${Wi}$ is the frequency of words, the equation below shows the probability that the word is female or not. 

<img src="files/image.png">

In [ ]:
model=LogisticRegressionCV(Cs=20,cv=5,penalty='l1',solver='liblinear',refit=True).fit(X_train,y_train)

coef=model.coef_
len(coef[0]) 
np.savetxt(dir_lasso+"coef_lasso_logit_full.txt",coef[0])

We will now predict the probability that the post is Female

In [ ]:
ypred_train=model.predict_proba(X_train)[:,1] # Pr(female=1)
ypred_test0=model.predict_proba(X_test0)[:,1]
ypred_test1=model.predict_proba(X_test1)[:,1]

"ypred" in the files are now brought back into gendered_posts.csv

In [ ]:
np.savetxt(dir_lasso+"ypred_train.txt",ypred_train)
np.savetxt(dir_lasso+"ypred_test0.txt",ypred_test0)
np.savetxt(dir_lasso+"ypred_test1.txt",ypred_test1)

# Logistic Classification Pronoun Sample Model

In [ ]:
model=LogisticRegressionCV(Cs=20,cv=5,penalty='l1',solver='liblinear',refit=True).fit(X_train,y_train)

coef=model.coef_
len(coef[0]) 
np.savetxt(dir_lasso+"coef_lasso_logit_pronoun.txt",coef[0])

We will now predict the probability that the post is Female

In [ ]:
ypred_train=model.predict_proba(X_train)[:,1] # Pr(female=1)
ypred_test0=model.predict_proba(X_test0)[:,1]
ypred_test1=model.predict_proba(X_test1)[:,1]

"ypred" in the files are now brought back into gendered_posts.csv

In [ ]:
np.savetxt(dir_lasso+"ypred_pronoun_train.txt",ypred_train)
np.savetxt(dir_lasso+"ypred_pronoun_test0.txt",ypred_test0)
np.savetxt(dir_lasso+"ypred_pronoun_test1.txt",ypred_test1)

# Future Exploration

If you are interested in Alice Wu's work, please check out her most recent paper titled "Gender Bias in Rumors among Professionals: An Identity-based Interpretation" which was published earlier this year in April. It can be found at https://scholar.harvard.edu/files/alicewu/files/ejr2019.pdf 